# Starting processes

In [120]:
from multiprocessing import Process
from multiprocessing import shared_memory
from time import sleep
import numpy as np
import numba as nb
import os

In [56]:
def compute():
    print(f"Hello! I am process number {os.getpid()}")
    print("Starting to work!")
    sleep(1)
    print("Done!")
    

def run_processes(n = 3):
    
    processes = []
    
    for i in range(n):
        print("Creating process", i)
        process = Process(target=compute)
        process.start()
        processes.append(process)
        
    # Waiting for all processees
    for i in range(n):
        processes[i].join()
        
    print("All the processes are done!")
        
%time run_processes(3)

Creating process 0
Creating process 1
Hello! I am process number 3769497
Creating process 2
Starting to work!
Hello! I am process number 3769502
Starting to work!
Hello! I am process number 3769511
Starting to work!
Done!
Done!
Done!
All the processes are done!
CPU times: user 4.72 ms, sys: 12.7 ms, total: 17.4 ms
Wall time: 1.02 s


In [87]:
@nb.njit()
def work_hard(n):
    r = 0
    for i in range(int(n)):
        r += i**0.5
        
    return r

def run_processes(todo=5_000_000, n = 3):
    
    processes = []
    
    for i in range(n):
        process = Process(target=work_hard, args=(todo/n, ))
        process.start()
        processes.append(process)
        
    # Waiting for all processees
    for i in range(n):
        processes[i].join()
        
    print("All the processes are done!")
        
        
%time run_processes(1_000_000_000, n=1)

All the processes are done!
CPU times: user 2.39 ms, sys: 1.5 ms, total: 3.89 ms
Wall time: 2.17 s


# Sharing memory

## Memmaps

In [117]:
def work(fname, i):
    output = np.lib.format.open_memmap(fname)
    output[i] += 1
    
def run_processes(matrix, n = 3):
    
    processes = []
    
    fname = '/tmp/tempmem.npy'
    output = np.lib.format.open_memmap(fname, shape=matrix.shape, dtype=matrix.dtype, mode='w+')
    output[:] = matrix
    
    for i in range(matrix.shape[0]):
        process = Process(target=work, args=(fname, i))
        process.start()
        processes.append(process)
        
    [p.join() for p in processes]
    
    return output
        
M = np.zeros((3, 5000, 5000))
%time result = run_processes(M)

CPU times: user 167 ms, sys: 254 ms, total: 422 ms
Wall time: 462 ms


In [123]:
def work(output, i):
    output[i] += 1
    
def run_processes(matrix, n = 3):
    
    processes = []
    
    fname = '/tmp/tempmem.npy'
    output = np.lib.format.open_memmap(fname, shape=matrix.shape, dtype=matrix.dtype, mode='w+')
    output[:] = matrix
    
    for i in range(matrix.shape[0]):
        process = Process(target=work, args=(output, i))
        process.start()
        processes.append(process)
        
    [p.join() for p in processes]
    
    return output
        
M = np.zeros((3, 5000, 5000))
%time result = run_processes(M)

CPU times: user 233 ms, sys: 181 ms, total: 415 ms
Wall time: 450 ms


## Shared Memory

In [122]:
def work(output, i):
    output[i] += 1
    
def run_processes(matrix, n = 3):
    
    processes = []
    
    for i in range(matrix.shape[0]):
        process = Process(target=work, args=(matrix, i))
        process.start()
        processes.append(process)
        
    [p.join() for p in processes]
        


M = np.zeros((3, 5000, 5000))
shm = shared_memory.SharedMemory(create=True, size=M.nbytes)
shm_arr = np.ndarray(M.shape, dtype=M.dtype, buffer=shm.buf)
shm_arr[:] = M

%time result = run_processes(M)


result = shm_arr.copy()
shm.close()
shm.unlink() # Delete region


CPU times: user 4.46 ms, sys: 3.74 ms, total: 8.2 ms
Wall time: 62.2 ms
